### Imports

In [1]:
from agents.fact_checker_agent import FactCheckerAgent
from network.socialnetwork import SocialNetworkEnv
from agents.news_agent import NewsAgent
import numpy as np

### Setup

In [ ]:
network = SocialNetworkEnv(numConsumer=50)

# created a fake and real info agent and add a few consumers to the network
fAgent = NewsAgent("fake-information", state_space_size=2, env=network, trustLevel=0)
fAgent2 = NewsAgent("fake-information", state_space_size=2, env=network, trustLevel=0)
fAgent3 = NewsAgent("fake-information", state_space_size=2, env=network, trustLevel=0)
rAgent = NewsAgent("real-information", state_space_size=2, env=network, trustLevel=0)
fcAgent = FactCheckerAgent("fact-checker", trustLevel=0.0, env=network)

network.add_news_agents_to_network(agentType=fAgent)
network.add_news_agents_to_network(agentType=fAgent2)
network.add_news_agents_to_network(agentType=fAgent3)
network.add_news_agents_to_network(agentType=rAgent)
network.add_fact_checker_to_network(agentType=fcAgent)

# print("---------------- Initial Graph ---------------")
network.print_graph()

Node 0: Points to -> 1
Node 1: Points to -> 0
Node 2: Points to -> 0, 1
Node 3: Points to -> 4, 2
Node 4: Points to -> 0, 2
Node 5: Points to -> 0, 1, 2, 3, 4
Node 6: Points to -> 0, 1, 2, 3, 4, 5
Node 7: Points to -> 0, 1, 2, 3, 4, 5, 6
Node 8: Points to -> 0, 1, 2, 3, 4
Node 9: Points to -> 0


### Training

In [3]:
for i in range(network.network_size):
    print(f"\n--------- epoch {i} ---------------------")

    fAgent_action = network.draw_sample_from_action_space()
    fAgent_action2 = network.draw_sample_from_action_space()
    fAgent_action3 = network.draw_sample_from_action_space()
    rAgent_action = network.draw_sample_from_action_space()


    print("Trust Levels:")
    print("Fake Agent: ", end="")
    reward, penalty = network.step(fAgent_action, fAgent)

    print("Fake Agent 2: ", end="")
    reward22, penalty22 = network.step(fAgent_action2, fAgent2)
    
    print("Fake Agent 3: ", end="")
    reward3, penalty3 = network.step(fAgent_action3, fAgent3)

    print("Real Agent: ", end="")
    reward2, penalty2 = network.step(rAgent_action, rAgent)

    network.step_fact_checker(fcAgent, threshold=0.1)

    # network.render()
    # network.print_graph()

    print(f"\nfake info - reward: {reward}, penalty : {penalty}")
    print(f"\nfake info 2 - reward: {reward22}, penalty : {penalty22}")
    print(f"\nfake info 3 - reward: {reward3}, penalty : {penalty3}")
    print(f"real info - reward: {reward2}, penalty : {penalty2}")
    print(f"fact checker - reward: {fcAgent.reward} penalty: {fcAgent.penalty}")



--------- epoch 0 ---------------------
Trust Levels:
Fake Agent: 0.2
Fake Agent 2: 0.2
Fake Agent 3: 0.2
Real Agent: 0.2
Agent type: fake-information penalty: 1 reward: 1 trust level: 0.2 0.2
Agent type: fake-information penalty: 0 reward: 1 trust level: 0.2 0.2
Agent type: fake-information penalty: 0 reward: 1 trust level: 0.2 0.2
Fact-checker penalized news agent fake-information.
Fact-checker penalized news agent fake-information.
Fact-checker penalized news agent fake-information.
set is  {Agent type: fake-information penalty: 1 reward: 1 trust level: 0.2, Agent type: fake-information penalty: 1 reward: 1 trust level: 0.2, Agent type: fake-information penalty: 2 reward: 1 trust level: 0.2}
before  [0.4, -0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
after  [-0.04999999999999999, -0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Fact-checker Q-value updated: 0.57

fake info - reward: 1, penalty : 1

fake info 2 - reward: 1, penalty : 0

fake info 3 - reward: 1, penalty : 0
real info - 